In [1]:
import mne 
import pickle
import argparse
import numpy as np
import os
import matplotlib.pyplot as plt

from src.params import SUBJ_CLEAN, RESULT_PATH, ACTIVE_RUN, PASSIVE_RUN, FIG_PATH, FREQS_LIST, PREPROC_PATH
from src.utils import get_bids_file
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import ShuffleSplit

from mne.decoding import (
    SlidingEstimator,
    GeneralizingEstimator,
    Scaler,
    cross_val_multiscore,
    LinearModel,
    get_coef,
    Vectorizer,
    CSP,
)

In [5]:
task = 'LaughterActive'
subj_list = SUBJ_CLEAN

if task == 'LaughterActive' :
    run_list = ACTIVE_RUN
elif task == 'LaughterPassive':
    run_list = PASSIVE_RUN

# Select what conditions to compute (str)
cond1 = 'LaughReal'
cond2 = 'LaughPosed'
freq_name = 'alpha'
conditions = cond1 + '-' + cond2
condition_list = [cond1, cond2]
picks = "meg" # Select MEG channels
time_window = True

tmin = 0
tmax = 0.5

TODO : 
Preprocessing : 
- Compute Power spectrum per oscillatory band
- Separate conditions

ML : 
- Use this function to select channel of interest : sklearn.feature_selection.SequentialFeatureSelector()
- Do the classification over time
- Plot the features of interest (topomaps)
- Plot the decoding accuracy over time

Repeat this for every oscillation

In [3]:
def time_window_hilbert(subj_list, RUN_LIST, tmin, tmax, FREQS_NAMES, FREQS_LIST) :
    
    epochs_list = []

    y_run_cond1 = []
    y_run_cond2 = []

    X_subj = []
    y_subj = []
    
    power_list = []
    FREQS = [x for x in range(len(FREQS_LIST))]
    idx = 0 #TODO : Change that

    for l_freq, h_freq in FREQS_LIST :
        print('Processing freqs -->', l_freq, h_freq)
        
        for subj in subj_list: # subj_list:
            print("=> Process task :", task, 'subject', subj)

            sub_id = 'sub-' + subj
            subj_path = os.path.join(RESULT_PATH, 'meg', 'reports', sub_id)

            if not os.path.isdir(subj_path):
                os.mkdir(subj_path)
                print('BEHAV folder created at : {}'.format(subj_path))
            else:
                print('{} already exists.'.format(subj_path))

            # Take ica data
            _, ica_path = get_bids_file(PREPROC_PATH, stage='ica', subj=subj)
            ica = mne.preprocessing.read_ica(ica_path)
        
            # Select rejection threshold associated with each run
            # TODO : apply threshold per run
            print('Load reject threshold')
            _, reject_path = get_bids_file(RESULT_PATH, stage="AR_epo", task=task, subj=subj, measure='log')

            with open(reject_path, 'rb') as f:
                reject = pickle.load(f)  

            # Select event_id appropriate
            for run in RUN_LIST: # RUN_LIST :
                print("=> Process run :", run)
                        
                if task == 'LaughterPassive' :
                    if run == '02' or run == '03' :
                        event_id = {'LaughReal' : 11, 'LaughPosed' : 12, 'EnvReal' : 21, 
                                    'EnvPosed' : 22}
                    elif run == '04' or run == '05' :
                        event_id = {'LaughReal' : 11, 'LaughPosed' : 12,'ScraReal' : 31, 
                                    'ScraPosed' : 32,}
                if task == 'LaughterActive' :
                        event_id = {'LaughReal' : 11, 'LaughPosed' : 12, 'Good' : 99, 'Miss' : 66}


                # Take raw data filtered : i.e. NO ICA 
                print('ICA')
                _, raw_path = get_bids_file(PREPROC_PATH, stage='proc-filt_raw', task=task, run=run, subj=subj)
                raw = mne.io.read_raw_fif(raw_path, preload=True)
                raw_filter = raw.copy()
                raw_filter = ica.apply(raw_filter)

                epochs_psds = []

                freq_name = FREQS_NAMES[idx]

                print('Apply filter')
                info = raw_filter.info
                raw_filter.filter(l_freq, h_freq) # Apply filter of interest
                raw_hilbert = raw_filter.apply_hilbert(envelope=True)

                picks = mne.pick_types(raw.info, meg=True, ref_meg=False, eeg=False, eog=False)

                # Segmentation
                print('Segmentation')
                events = mne.find_events(raw)
                epochs = mne.Epochs(
                    raw_hilbert,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=(0, 0),
                    picks=picks)    

                # Auto-reject epochs
                epochs.drop_bad(reject=reject)

                if subj == subj_list[0] and run == run_list[0]:
                    head_info = epochs.info['dev_head_t']
                else:
                    epochs.info['dev_head_t'] = head_info

                epochs.equalize_event_counts(['LaughReal', 'LaughPosed'])
                epochs_list.append(epochs)
                del epochs
                
        epochs_all_run = mne.concatenate_epochs(epochs_list)  

        # Compute power (= envelop **2)
        print('Calculte power')
        power_cond1 = epochs_all_run[cond1].get_data()**2 # Shape n_epochs, n_chan, n_times
        power_cond2 = epochs_all_run[cond2].get_data()**2 # Shape n_epochs, n_chan, n_times

        power_cond1_mean = np.mean(power_cond1, axis=2) # Mean across time
        power_cond2_mean = np.mean(power_cond2, axis=2)
        
        X_subj.append(np.concatenate((power_cond1, power_cond2)))
        print('Concatenation')
        X_subj.append(np.concatenate((power_cond1_mean, power_cond2_mean)))
        y_subj.append(np.concatenate((epochs_all_run[cond1].events[:, 2], epochs_all_run[cond2].events[:, 2])))

        print(X_subj[-1].shape)
        print(y_subj[-1].shape)
        
    return X_subj, y_subj

In [5]:
# Change this params
epochs_list = []

y_run_cond1 = []
y_run_cond2 = []

X_subj = []
y_subj = []

subj_list = SUBJ_CLEAN
run_list = run_list

if time_window == True :
    X_subj, y_subj = time_window_hilbert(subj_list, run_list, tmin, tmax, FREQS_NAMES=[freq_name], FREQS_LIST=[FREQS_LIST[2]])
    
else : 
    for subj in subj_list: 

        print(subj)

        for run in run_list :

            # importer les psd (enveloppe spectrale)
            _, path_epochs = get_bids_file(RESULT_PATH, task=task, subj=subj, run=run, measure=freq_name, stage="psd_epo")
            epochs = mne.read_epochs(path_epochs, verbose=None)

            epochs.apply_baseline(baseline=(None, 0))
            epochs.resample(50)

            if subj == subj_list[0] and run == run_list[0]:
                head_info = epochs.info['dev_head_t']
            else:
                epochs.info['dev_head_t'] = head_info

            epochs_list.append(epochs)

        epochs_all_run = mne.concatenate_epochs(epochs_list)  

        # Compute power (= envelop **2)
        power_cond1 = epochs_all_run[cond1].get_data()**2 # Shape n_epochs, n_chan, n_times
        power_cond2 = epochs_all_run[cond2].get_data()**2 # Shape n_epochs, n_chan, n_times
        
        X_subj.append(np.concatenate((power_cond1, power_cond2)))
        y_subj.append(np.concatenate((epochs_all_run[cond1].events[:, 2], epochs_all_run[cond2].events[:, 2])))


Processing freqs --> 8 12
=> Process task : LaughterActive subject 01
/home/claraelk/scratch/laughter_data/results/meg/reports/sub-01 already exists.
Reading /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_ica.fif ...
Now restoring ICA solution ...
Ready.
Load reject threshold
=> Process run : 07
ICA
Opening raw data file /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_run-07_proc-filt_raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 299999 =      0.000 ...   249.999 secs
Ready.
Current compensation grade : 3
Reading 0 ... 299999  =      0.000 ...   249.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (189 components)
    Zeroing out 8 ICA components
    Projecting back using 273 PCA components
Apply filter
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 12 Hz

FIR filter para

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 302 out of 302 | elapsed:    2.2s finished


Segmentation
Trigger channel has a non-zero initial value of 68 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
74 events found
Event IDs: [ 5 11 12 66 99]
Not setting metadata
49 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 49 events and 601 original time points ...
0 bad epochs dropped
Dropped 5 epochs: 0, 1, 3, 5, 15
=> Process run : 08
ICA
Opening raw data file /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_run-08_proc-filt_raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 299999 =      0.000 ...   249.999 secs
Ready.
Current compensation grade : 3
Reading 0 ... 299999  =      0.000 ...   249.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (189 com

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 302 out of 302 | elapsed:    2.2s finished


Segmentation
Trigger channel has a non-zero initial value of 68 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
75 events found
Event IDs: [ 5 11 12 66 99]
Not setting metadata
50 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 50 events and 601 original time points ...
0 bad epochs dropped
Dropped 5 epochs: 0, 2, 4, 18, 20
=> Process run : 09
ICA
Opening raw data file /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_run-09_proc-filt_raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 299999 =      0.000 ...   249.999 secs
Ready.
Current compensation grade : 3
Reading 0 ... 299999  =      0.000 ...   249.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (189 co

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 302 out of 302 | elapsed:    7.5s finished


Segmentation
Trigger channel has a non-zero initial value of 68 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
75 events found
Event IDs: [ 5 11 12 66 99]
Not setting metadata
50 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 50 events and 601 original time points ...
0 bad epochs dropped
Dropped 5 epochs: 0, 2, 20, 22, 28


/tmp/ipykernel_2202987/3713707931.py:99: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_all_run = mne.concatenate_epochs(epochs_list)


Using data from preloaded Raw for 44 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 44 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...

Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...
Using data from preloaded Raw for 45 events and 601 original time points ...

In [6]:
len(X_subj)

2

In [7]:
X = X_subj[-1]
X.shape

(3240, 270)

In [8]:
y = y_subj[-1]
y.shape

(3240,)

In [6]:
stage = 'ml-'+ str(tmin) + '-' + str(tmax)
sub_name = 'all'
run_name = 'all'

save_X = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                    run_name, conditions,
                                                                                    'X-subj', stage)
save_y = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                    run_name, conditions,
                                                                                    'y-subj', stage)
with open(save_X, 'rb') as f:
    X_subj = pickle.load(f)
    
with open(save_y, 'rb') as f:
    y_subj = pickle.load(f)

'''
with open(save_X, 'wb') as f:
    pickle.dump(X_subj, f)
    
with open(save_y, 'wb') as f:
    pickle.dump(y_subj, f)'''

"\nwith open(save_X, 'wb') as f:\n    pickle.dump(X_subj, f)\n    \nwith open(save_y, 'wb') as f:\n    pickle.dump(y_subj, f)"

## ML single feature
- with time windows

In [23]:
X_subj[-1]

array([[5.94740356e-29, 1.78025982e-28, 3.59543638e-28, ...,
        1.09550452e-27, 4.28214818e-28, 9.61208219e-28],
       [1.25423664e-28, 6.82147481e-28, 7.93367080e-28, ...,
        1.64309339e-27, 4.37999346e-28, 8.03752513e-27],
       [5.84094505e-28, 2.75852338e-28, 5.52373637e-28, ...,
        2.93682159e-27, 2.97504642e-28, 3.95165169e-27],
       ...,
       [9.63082510e-28, 1.01524959e-27, 4.07652755e-28, ...,
        2.21633269e-27, 2.53780747e-28, 4.33161435e-27],
       [7.30789622e-28, 5.99786668e-28, 3.94082974e-28, ...,
        3.56224725e-28, 1.21962809e-28, 4.26567881e-28],
       [1.46809507e-27, 1.17572583e-27, 9.77154581e-28, ...,
        1.69804207e-27, 2.22050058e-28, 4.86145541e-27]])

In [30]:
CHAN = np.arange(0, 270, 25)

scores_chan = []

# Select the classifier & cross-val method
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    
X = X_subj[-1]
print(X.shape)

y = y_subj[-1]
print(y.shape)

X_random = np.random.rand(3240, 10)


for chan in CHAN :
    print('Process channel :', chan)
    clf = svm.SVC(kernel='linear', C=1, random_state=42)

    # Select channel of interest
    X_chan = X[:, chan]
    X_classif = X_chan.reshape(-1, 5) # Reshape (n_event, 1)
    print(X_chan.shape)
    print(X_classif.shape)
    
    #X_random_chan = X_random[:, chan]
    #X_random_chan = X_random_chan.reshape(-1, 1)
    
    scores = cross_val_score(clf, X_classif, y, cv=cv)
    print(scores)
    
    # Mean scores across cross-validation splits
    scores_mean = np.mean(scores, axis=0)
    print('Score :', scores_mean)

    # Put results into list
    scores_chan.append(scores_mean)

# Put all scores into numpy
arr_scores = np.array(scores_chan)
    

(3240, 270)
(3240,)
Process channel : 0
(3240, 5)
(3240, 5)
[0.47530864 0.48868313 0.49794239 0.46399177 0.49382716]
Score : 0.4839506172839506
Process channel : 25
(3240, 5)
(3240, 5)
[0.47530864 0.48868313 0.49794239 0.46399177 0.49382716]
Score : 0.4839506172839506
Process channel : 50
(3240, 5)
(3240, 5)
[0.47530864 0.48868313 0.49794239 0.46399177 0.49382716]
Score : 0.4839506172839506
Process channel : 75
(3240, 5)
(3240, 5)
[0.47530864 0.48868313 0.49794239 0.46399177 0.49382716]
Score : 0.4839506172839506
Process channel : 100
(3240, 5)
(3240, 5)


KeyboardInterrupt: 

In [26]:
X_classif.shape

(3240, 1)

In [18]:
X[:, 0] == X[:, 1]

array([False, False, False, ..., False, False, False])

In [91]:
X_chan.shape
a = X_chan.reshape(-1, 1)
a.shape

(675, 1)

## Multi-features
- Decoding over-time

In [ ]:
# We will train the classifier on all left visual vs auditory trials on MEG
# TODO : add feature selection (Channel selection)
sfs = SequentialFeatureSelector(classifier, n_features_to_select=60)

clf = make_pipeline(StandardScaler(), sfs, classifier)

time_decod = SlidingEstimator(clf, scoring="roc_auc", verbose=True, n_jobs=-1)
# here we use cv=3 just for speed
scores = cross_val_multiscore(time_decod, X, y, cv=3, n_jobs=-1)

# Mean scores across cross-validation splits
scores = np.mean(scores, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(epochs.times, scores, label="score")
ax.axhline(0.5, color="k", linestyle="--", label="chance")
ax.set_xlabel("Times")
ax.set_ylabel("AUC")  # Area Under the Curve
ax.legend()
ax.axvline(0.0, color="k", linestyle="-")
ax.set_title("Sensor space decoding")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.

  0%|          | Fitting SlidingEstimator : 0/100 [00:00<?,       ?it/s]

  0%|          | Fitting SlidingEstimator : 0/100 [00:00<?,       ?it/s]